<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine%20v7.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# requirements
import nltk
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('colheader_justify', 'left')
import requests
import operator
import flask
from keras.preprocessing.text import Tokenizer
import re
import string
from wordcloud import STOPWORDS
! pip install pyspellchecker
from spellchecker import SpellChecker
from gensim.models.keyedvectors import KeyedVectors
import warnings
warnings.filterwarnings('ignore')


Using TensorFlow backend.


    100% |████████████████████████████████| 1.9MB 16.8MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CCP-ICDsearch"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
CD.npy	ICD_dict.npy  map_abbreviation.npy  wikipedia-pubmed-and-PMC-w2v.bin


In [0]:
# Functions
def clean(text):
  x = wpt.tokenize(text.lower())
  x = [w for w in x if w not in list(STOPWORDS)+['(',')','-',"'",':',',','.','{','}','_']]
  x = [map_abbreviation[word] if word in list(map_abbreviation.keys()) else word for word in x]
  x = list(set([spell.correction(word) if word not in model.wv.vocab else word for word in x ]))
  vect = np.array([np.zeros((200,)) if word not in model.wv.vocab else model.wv[word] for word in x])
  return vect

In [0]:
# Initializers

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)
wpt = nltk.WordPunctTokenizer()
spell = SpellChecker()


In [0]:
# KB
def load_KB():
    ICD_dict = np.load('/content/drive/My Drive/CCP-ICDsearch/ICD_dict.npy')
    ICD_dict = sorted(dict(ICD_dict).items(), key=lambda kv: kv[0])
    map_abbreviation = np.load('/content/drive/My Drive/CCP-ICDsearch/map_abbreviation.npy').item()
    CD=np.load('/content/drive/My Drive/CCP-ICDsearch/CD.npy')
    return ICD_dict, CD, map_abbreviation
  

In [0]:
def get_result(search_term):
    DD = clean(search_term)
    out = np.dot(np.mean(CD,axis=1),(np.mean(DD,axis=0)).T)
    #d1 = sum(np.diagonal(np.dot(DD,DD.T)))
    #d2 = np.count_nonzero(np.sum(CD, axis=2), axis=1) + 0.00001
    #out = n/(d1*d2)
    index = np.argsort(out)[-7:]
    scores = out[index[::-1]]
    result = np.array(ICD_dict)[index[::-1]]
    df = pd.DataFrame(result,scores*100)
    df.reset_index(level=0, inplace=True)
    df.columns = ['Confidence','ICD-10','Description']
    df['Confidence']=df['Confidence'].apply(lambda x: round(x,2))
    return df

In [18]:
#initialise()
ICD_dict, CD, map_abbreviation = load_KB()
df = get_result("catract in left eye")
df

,Confidence,ICD-10,Description
0,176.79,H59022,"Cataract (lens) fragments in eye following cataract surgery, left eye"
1,175.97,H59021,"Cataract (lens) fragments in eye following cataract surgery, right eye"
2,158.16,M23212,"Derangement of anterior horn of medial meniscus due to old tear or injury, left knee"
3,157.63,M23242,"Derangement of anterior horn of lateral meniscus due to old tear or injury, left knee"
4,157.34,M23211,"Derangement of anterior horn of medial meniscus due to old tear or injury, right knee"
5,156.81,M23241,"Derangement of anterior horn of lateral meniscus due to old tear or injury, right knee"
6,155.34,H59032,"Cystoid macular edema following cataract surgery, left eye"
